#### -----------------------------------------------------------------------------<br>Copyright (c) 2024, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

#### Enumeration: CcpSwitchover
>    This example introduces device enumeration with the ability to hand over control
    to another process. This includes opening and closing the system, updating and 
    retrieving the list of devices, searching for devices, and creating and destroying
    a device. In this example, we will also set a special key to the device that another 
    process can use to aquire control of the device when running the example for the first
    time. Running the example a second time while the first instance is still running will 
    try to use the special key to gain control of the device. 


In [1]:
from arena_api.system import system
import time

TAB1 = "  "
TAB2 = "    "

In [2]:
'''
Create devices
'''

tries = 0
tries_max = 6
sleep_time_secs = 10
device_infos = None

while tries < tries_max:  # Wait for device for 60 seconds
    device_infos = system.device_infos
    if not device_infos:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1} seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(device_infos)} device(s)\n')
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

Created 1 device(s)



In [3]:
def has_control(device):
	'''
	Check if device has control
	'''
	device_access_status = device.tl_device_nodemap['DeviceAccessStatus'].value
	return device_access_status == "ReadWrite"

##### enumerates device(s)
Demonstrated enumeration
> - gets device list
> - save first serial number to demonstrate search
> - prints device information
> - demonstrates device search
> - create device
> - if running first instance will set a special key
> - if running second instance will use key to aquire control
> - cleans up

In [4]:
serial_to_find = device_infos[0]['serial']
serial_to_find

'420'

In [5]:
for i, device_info in enumerate(device_infos): 
    print(f"Information for device {i}:")
    vendor = device_info['vendor']
    model = device_info['model']
    serial = device_info['serial']
    macStr = device_info['mac']
    ipStr = device_info['ip']
    print(f"{TAB1}Vendor: {vendor}; model: {model}; serial: {serial}; MAC: {macStr}; ip: {ipStr}")

Information for device 0:
  Vendor: Lucid Vision Labs; model: PHX050S-C; serial: 420; MAC: 1c:0f:af:12:53:65; ip: 169.254.3.2


In [6]:
matching_device_info = None

print(f"Searching for device with serial number: {serial_to_find}")
for device_info in device_infos:
    if device_info['serial'] == serial_to_find:
        matching_device_info = device_info
        break

if matching_device_info:
    print(f"Device found!")
    '''
    Create device
        Create device in order to configure it and grab images. A device can only be created
        once per process, and can only be opened with read-write access once. 
    '''

    print(f"Creating device")
    devices = system.create_device(matching_device_info)
    device = devices[0]

    # static key used to acquire control between applications
    switchoverKey = 0x1234
    device.tl_device_nodemap['CcpSwitchoverKey'].value = switchoverKey
    
    if has_control(device):
        print(f'Set the switchover key. Continue')
    else:
        device.tl_device_nodemap['DeviceAccessStatus'].value = "ReadWrite"
        if has_control(device):
            print(f"Create device succeeded with acquiring control")
        else:
            print(f"Create device failed to acquire control")
            
    system.destroy_device()
    print(f"Destroyed all created devices")

Searching for device with serial number: 420
Device found!
Creating device
Set the switchover key. Continue
Destroyed all created devices


##### Clean up ----------------------------------------------------------------
> Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [7]:
system.destroy_device()